final

In [ ]:
## datadam_vacant_price_score


## 확인사항
## 1. KT데이터 업데이트 여부
## 2. 건축물대장 업데이트 여부
## 3. 실거래가 업데이트 여부
## 4. 한국부동산원 지역별공실률 업데이트 여부

# module
import psycopg2
import pandas as pd

# 기준월
# date = '202211'
dates = pd.date_range('2022-01',periods=10,freq='M').strftime("%Y%m")
quarter = '202202'

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 시군구코드
aws_cursor.execute(
"""
select distinct substring(pnu, 1, 5) as sig_cd
from m1.pnu
order by 1
"""
)
aws_conn.commit()
rows = aws_cursor.fetchall()
df = pd.DataFrame(rows, columns = ['sig_cd'])
lawd_cd = list(df['sig_cd'])


# INSERT
for date in dates:
    for i in lawd_cd:
        aws_cursor.execute(
        """
        insert into m2.datadam_vacant_price_score (
        with date as (
            select '""" + date + """'::text as base_ym
        )
        select (select base_ym from date) as base_ym
             , A.pnu
             , substring(A.pnu, 1, 8) as emd_cd
             , case when
                    round(
                            (
                                nvl((avg(B.vac_rate) - avg(C.vac_rate_avg)) / avg(C.vac_rate_std) * 5 + 75, 75) +
                                nvl((avg(D.deal_amount_per_area) - avg(E.deal_amount_avg)) / avg(E.deal_amount_std) * 5 + 75, 75)
                            ) / 2, 2
                        ) > 100 then 100
                    else
                    round(
                            (
                                nvl((avg(B.vac_rate) - avg(C.vac_rate_avg)) / avg(C.vac_rate_std) * 5 + 75, 75) +
                                nvl((avg(D.deal_amount_per_area) - avg(E.deal_amount_avg)) / avg(E.deal_amount_std) * 5 + 75, 75)
                            ) / 2, 2
                        ) end as score
             , sysdate::date as create_at
             , sysdate::date as update_at
             , 'du.Park' as work_user
        from (
            -- 모집단 가져오기 : KT매출 & PNU존재 & 건물존재
            select T1.pnu
                 , ST_Centroid(T2.poly::geometry) as poly
            from (
                -- KT매출 존재
                select distinct substring(bd_mgt_sn, 1, 19) as pnu
                from m1.kt_bldg_sales
                where 1=1
                    and base_ym = (select base_ym from date) -- 기준월
                    and substring(bd_mgt_sn, 1, 5) = '""" + str(i) + """' -- 시군구
            ) T1
            -- PNU 존재
            join m2.cremao_land T2
                on T1.pnu = T2.pnu
            -- 건물 존재
            join (
                select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
                from m1.bld_rgst
                where base_ym = (select base_ym from date) -- 기준월
            ) T3
                on T1.pnu = T3.pnu
        ) A
        left join (
            select *
            from m1.vacant_rate
            where research_date = '""" + quarter + """' -- 기준분기
        ) B
            on substring(A.pnu, 1, 7) = B.region_cd
        left join (
            select avg(vac_rate) as vac_rate_avg
                 , stddev(vac_rate) as vac_rate_std 
            from m1.vacant_rate
            where 1=1
                and research_date = '""" + quarter + """' -- 기준분기
                and substring(region_cd, 1, 5) = '""" + str(i) + """' -- 시군구
            group by 1
        ) C
            on 1=1
        left join (
            select ST_Centroid(T2.poly::geometry) as poly
                 , round(1.0 * T1.deal_amount * 10000 / T1.building_area, 0) as deal_amount_per_area
            from m2.cremao_building_trade_case T1
            left join m2.cremao_land T2
                on T1.pnu = T2.pnu
            where 1=1
                and T1.contract_year_month between (select dateadd(month, -59, to_date(base_ym||'01', 'yyyymmdd'))::date from date) and (select last_day(to_date(base_ym||'01', 'yyyymmdd')) from date) -- 최근 3년
                and substring(T1.pnu, 1, 5) = '""" + str(i) + """' -- 시군구
        ) D
            on ST_DistanceSphere(A.poly, D.poly) < 150 -- 반경 150m PNU
        left join (
            select round(avg(1.0 * deal_amount * 10000 / building_area), 0) as deal_amount_avg
                 , round(stddev(1.0 * deal_amount * 10000 / building_area), 0) as deal_amount_std
            from m2.cremao_building_trade_case
            where 1=1
                and contract_year_month between (select dateadd(month, -59, to_date(base_ym||'01', 'yyyymmdd'))::date from date) and (select last_day(to_date(base_ym||'01', 'yyyymmdd')) from date) -- 최근 3년
                and substring(pnu, 1, 5) = '""" + str(i) + """' -- 시군구
        ) E
            on 1=1
        group by 1, 2, 3
        );
        """
        )
        aws_conn.commit()
        print(i, "Done")

###### ver1

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 평균 연면적
sql = '''
select sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu, avg(tot_area) as avg_area
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
 group by sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['pnu','area_avg']
df_bld_rgst['emd_cd'] = df_bld_rgst['pnu'].str[:10]

# pnu별 주용도명
sql = '''
select main_purps_cd_nm, sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_principal_use = pd.DataFrame(rows)
df_bld_principal_use.columns = ['principal_use','pnu']

# 법정동별 평균 매출액
sql = '''
select lgl_dong_cd||'00' as emd_cd, avg(avg_biz_income) as sales_avg
  from m1.kcb_soho	 
 group by lgl_dong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kcb_soho = pd.DataFrame(rows)
df_kcb_soho.columns = ['emd_cd','sales_avg']
df_kcb_soho['sales_avg'] = (df_kcb_soho['sales_avg']*1000).astype(float) # 금액 : 원(won) 단위

In [ ]:
# 데이터 프레임 병합
df = pd.merge(df_kcb_soho, df_bld_rgst, on='emd_cd', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['area_avg'].isnull()]

# 매출액 / 면적
df['sales_per_area'] = (df['sales_avg'] / df['area_avg'])

# 주용도코드 붙이기
df = pd.merge(df, df_bld_principal_use, on='pnu', how='left')
df = df.drop_duplicates('pnu').reset_index(drop=True)

# inf 값 제거후 컬럼 정리
df = df[~(df['sales_per_area']==np.inf)]
df = df[['pnu','emd_cd','principal_use','sales_per_area']]
df['base_ym'] = datetime.today().strftime("%Y%m")

In [ ]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_vacant_price_score'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_sales_per_area;
    create table m2.datadam_sales_per_area (    
    pnu varchar(19)
    , emd_cd varchar(10)
    , principal_use varchar(200)
    , sales_per_area double precision
    , base_ym varchar(6)
    )
    distkey(emd_cd)
    compound sortkey(base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))